In [1]:
import os
import tensorflow as tf
import dask
import dask.array as da
import numpy as np

In [2]:
train_x = da.from_npy_stack('/home/skyolia/JupyterProjects/classification/cifar_10/data/train_x')
train_y = da.from_npy_stack('/home/skyolia/JupyterProjects/classification/cifar_10/data/train_y')
test_x = da.from_npy_stack('/home/skyolia/JupyterProjects/classification/cifar_10/data/test_x')
test_y = da.from_npy_stack('/home/skyolia/JupyterProjects/classification/cifar_10/data/test_y')
train_x, train_y, test_x, test_y

(dask.array<from-npy-stack, shape=(50000, 32, 32, 3), dtype=float64, chunksize=(10000, 32, 32, 3)>,
 dask.array<from-npy-stack, shape=(50000, 1), dtype=float64, chunksize=(10000, 1)>,
 dask.array<from-npy-stack, shape=(10000, 32, 32, 3), dtype=float64, chunksize=(1000, 32, 32, 3)>,
 dask.array<from-npy-stack, shape=(10000, 1), dtype=float64, chunksize=(1000, 1)>)

In [3]:
class CIFAR10Sequence(tf.keras.utils.Sequence):

    def __init__(self, x, y, batch_size):
        self.x, self.y = x, y
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        return batch_x, batch_y
    
class SGDR(tf.keras.callbacks.Callback): 

    def __init__(self, on_start_lenght, min_lr, max_lr, cycle_length, steps_per_epoch, mult_factor=1, lr_decay=1):
        self.on_start_lenght = on_start_lenght
        self.min_lr = min_lr
        self.max_lr = max_lr
        self.cycle_length = cycle_length
        self.steps_per_epoch = steps_per_epoch
        #self.next_restart = cycle_length
        self.mult_factor = mult_factor
        self.lr_decay = lr_decay
        self.batch_since_restart = 0
        self.batch_counter = 0
        
    def update_lr(self): #update learning rate after each batch
        fraction_to_restart = self.batch_since_restart / (self.steps_per_epoch * self.cycle_length)
        lr = self.min_lr + 0.5 * (self.max_lr - self.min_lr) * (1 + np.cos(fraction_to_restart * np.pi))
        return lr
    
    def on_batch_end(self, batch, logs=None):
        logs = logs or {}
        self.batch_counter += 1
        if self.batch_counter + 1 >= self.on_start_lenght * self.steps_per_epoch:
            self.batch_since_restart += 1
            tf.keras.backend.set_value(self.model.optimizer.lr, self.update_lr())
        
    def on_epoch_end(self, epoch, logs={}):
        if epoch + 1 <= self.on_start_lenght:
            self.next_restart = self.cycle_length + self.on_start_lenght
        if epoch + 1 == self.next_restart:
            self.batch_since_restart = 0
            self.cycle_length = np.ceil(self.cycle_length * self.mult_factor)
            self.next_restart += self.cycle_length
            self.max_lr *= self.lr_decay
        
class LRTensorBoard(tf.keras.callbacks.TensorBoard):
    '''
    Add learning rate evolution to Tensorboard
    '''
    def __init__(self, log_dir):
        super().__init__(log_dir=log_dir)

    def on_epoch_end(self, epoch, logs=None):
        logs['lr'] = tf.keras.backend.eval(self.model.optimizer.lr)
        super().on_epoch_end(epoch, logs)


def build_block(input_layer, filters, norm=True, k=[3,3]):
    layer = tf.keras.layers.Conv2D(filters, kernel_size=(k[0], k[1]), padding='same', use_bias=not norm, kernel_initializer='glorot_uniform')(input_layer)
    if norm:
        layer = tf.keras.layers.BatchNormalization()(layer)
    layer = tf.keras.layers.Activation('elu')(layer)
    return layer

def build_model(num_class):
    image_input = tf.keras.Input(shape=(32, 32, 3), name='input_layer')
    conv_1 = build_block(image_input, 48)
    conv_2 = build_block(conv_1, 48)
    pool_1 = tf.keras.layers.MaxPooling2D(padding='same')(conv_2)
    drop_1 = tf.keras.layers.Dropout(0.6)(pool_1)
    conv_3 = build_block(drop_1, 96)
    conv_4 = build_block(conv_3, 96)
    pool_2 = tf.keras.layers.MaxPooling2D(padding='same')(conv_4)
    drop_2 = tf.keras.layers.Dropout(0.6)(pool_2)
    conv_5 = build_block(drop_2, 192)
    conv_6 = build_block(conv_5, 192)
    pool_3 = tf.keras.layers.MaxPooling2D(padding='same')(conv_6)
    drop_3 = tf.keras.layers.Dropout(0.6)(pool_3)
    conv_7 = build_block(drop_3, 192, False, [1,1])
    drop_4 = tf.keras.layers.Dropout(0.25)(conv_7)
    gap = tf.keras.layers.GlobalAvgPool2D()(drop_4)
    logits = tf.keras.layers.Dense(units=num_class, activation='softmax', bias_initializer='ones', kernel_initializer='glorot_uniform')(gap)
    model = tf.keras.Model(inputs=image_input, outputs=logits)
    return model

In [4]:
model = build_model(10)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 32, 32, 48)        1296      
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 48)        192       
_________________________________________________________________
activation (Activation)      (None, 32, 32, 48)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 48)        20736     
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 48)        192       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 48)        0         
__________

In [5]:
model.load_weights("day_2.weights.best.hdf5")
lr=0.001
adam = tf.keras.optimizers.Adam(lr=lr)
model.compile(loss='sparse_categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
scores = model.evaluate(test_x, test_y)
scores

10000/10000 [==============================] - 5s 451us/step


[0.385229864937067, 0.899]

In [5]:
epochs, min_lr, max_lr, batch_size = 10000, 0.002, 0.1, 256
steps_per_epoch = int(np.ceil(train_y.shape[0]/batch_size))
filepath="day_1.weights.best.hdf5"

train_generator = CIFAR10Sequence(train_x, train_y, batch_size)
test_generator = CIFAR10Sequence(test_x, test_y, batch_size)
tblr = LRTensorBoard(log_dir=os.getcwd())
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
sgdr_lr = SGDR(100, min_lr, max_lr, 250, steps_per_epoch)

sgd = tf.keras.optimizers.SGD(lr=max_lr, momentum=0.9)
model.compile(loss='sparse_categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#model.load_weights(filepath)
model.fit_generator(generator=train_generator,
                    steps_per_epoch=steps_per_epoch,
                    epochs=epochs,
                    verbose=1,
                    validation_data=test_generator,
                    validation_steps=int(np.ceil(test_y.shape[0]/batch_size)),
                    use_multiprocessing=True,
                    workers=12,
                    shuffle=True,
                    #initial_epoch=600,
                    callbacks=[checkpoint, sgdr_lr, tblr])


Epoch 1/10000
195/196 [============================>.] - ETA: 0s - loss: 1.7530 - acc: 0.3612
Epoch 00001: val_acc improved from -inf to 0.23530, saving model to day_1.weights.best.hdf5
196/196 [==============================] - 92s 470ms/step - loss: 1.7513 - acc: 0.3620 - val_loss: 2.2282 - val_acc: 0.2353
Epoch 2/10000
195/196 [============================>.] - ETA: 0s - loss: 1.3493 - acc: 0.5037
Epoch 00002: val_acc improved from 0.23530 to 0.50850, saving model to day_1.weights.best.hdf5
196/196 [==============================] - 32s 162ms/step - loss: 1.3490 - acc: 0.5040 - val_loss: 1.3601 - val_acc: 0.5085
Epoch 3/10000
195/196 [============================>.] - ETA: 0s - loss: 1.1771 - acc: 0.5744
Epoch 00003: val_acc improved from 0.50850 to 0.55400, saving model to day_1.weights.best.hdf5
196/196 [==============================] - 32s 162ms/step - loss: 1.1773 - acc: 0.5744 - val_loss: 1.2816 - val_acc: 0.5540
Epoch 4/10000
195/196 [============================>.] - ETA: 0s

Epoch 29/10000
195/196 [============================>.] - ETA: 0s - loss: 0.5612 - acc: 0.8025
Epoch 00029: val_acc did not improve from 0.82270
196/196 [==============================] - 33s 170ms/step - loss: 0.5609 - acc: 0.8024 - val_loss: 0.5396 - val_acc: 0.8112
Epoch 30/10000
195/196 [============================>.] - ETA: 0s - loss: 0.5478 - acc: 0.8085
Epoch 00030: val_acc did not improve from 0.82270
196/196 [==============================] - 33s 170ms/step - loss: 0.5482 - acc: 0.8084 - val_loss: 0.5416 - val_acc: 0.8131
Epoch 31/10000
195/196 [============================>.] - ETA: 0s - loss: 0.5449 - acc: 0.8066
Epoch 00031: val_acc did not improve from 0.82270
196/196 [==============================] - 33s 169ms/step - loss: 0.5455 - acc: 0.8063 - val_loss: 0.5337 - val_acc: 0.8214
Epoch 32/10000
195/196 [============================>.] - ETA: 0s - loss: 0.5376 - acc: 0.8114
Epoch 00032: val_acc improved from 0.82270 to 0.82290, saving model to day_1.weights.best.hdf5
196

195/196 [============================>.] - ETA: 0s - loss: 0.4109 - acc: 0.8557
Epoch 00058: val_acc did not improve from 0.85760
196/196 [==============================] - 33s 169ms/step - loss: 0.4113 - acc: 0.8556 - val_loss: 0.4746 - val_acc: 0.8463
Epoch 59/10000
195/196 [============================>.] - ETA: 0s - loss: 0.4061 - acc: 0.8569
Epoch 00059: val_acc did not improve from 0.85760
196/196 [==============================] - 34s 172ms/step - loss: 0.4060 - acc: 0.8570 - val_loss: 0.4562 - val_acc: 0.8505
Epoch 60/10000
195/196 [============================>.] - ETA: 0s - loss: 0.4048 - acc: 0.8552
Epoch 00060: val_acc did not improve from 0.85760
196/196 [==============================] - 34s 172ms/step - loss: 0.4052 - acc: 0.8551 - val_loss: 0.4670 - val_acc: 0.8489
Epoch 61/10000
195/196 [============================>.] - ETA: 0s - loss: 0.4057 - acc: 0.8570
Epoch 00061: val_acc did not improve from 0.85760
196/196 [==============================] - 34s 172ms/step - los

195/196 [============================>.] - ETA: 0s - loss: 0.3349 - acc: 0.8822
Epoch 00087: val_acc did not improve from 0.87390
196/196 [==============================] - 33s 168ms/step - loss: 0.3346 - acc: 0.8824 - val_loss: 0.4427 - val_acc: 0.8583
Epoch 88/10000
195/196 [============================>.] - ETA: 0s - loss: 0.3316 - acc: 0.8834
Epoch 00088: val_acc did not improve from 0.87390
196/196 [==============================] - 33s 169ms/step - loss: 0.3318 - acc: 0.8832 - val_loss: 0.3961 - val_acc: 0.8718
Epoch 89/10000
195/196 [============================>.] - ETA: 0s - loss: 0.3305 - acc: 0.8819
Epoch 00089: val_acc did not improve from 0.87390
196/196 [==============================] - 33s 168ms/step - loss: 0.3302 - acc: 0.8820 - val_loss: 0.4638 - val_acc: 0.8577
Epoch 90/10000
195/196 [============================>.] - ETA: 0s - loss: 0.3304 - acc: 0.8828
Epoch 00090: val_acc did not improve from 0.87390
196/196 [==============================] - 33s 168ms/step - los

Epoch 117/10000
195/196 [============================>.] - ETA: 0s - loss: 0.2844 - acc: 0.8993
Epoch 00117: val_acc did not improve from 0.88470
196/196 [==============================] - 34s 176ms/step - loss: 0.2845 - acc: 0.8994 - val_loss: 0.4442 - val_acc: 0.8672
Epoch 118/10000
195/196 [============================>.] - ETA: 0s - loss: 0.2806 - acc: 0.9010
Epoch 00118: val_acc did not improve from 0.88470
196/196 [==============================] - 34s 174ms/step - loss: 0.2807 - acc: 0.9009 - val_loss: 0.4324 - val_acc: 0.8626
Epoch 119/10000
195/196 [============================>.] - ETA: 0s - loss: 0.2818 - acc: 0.8991
Epoch 00119: val_acc did not improve from 0.88470
196/196 [==============================] - 34s 173ms/step - loss: 0.2815 - acc: 0.8992 - val_loss: 0.4215 - val_acc: 0.8723
Epoch 120/10000
195/196 [============================>.] - ETA: 0s - loss: 0.2818 - acc: 0.8993
Epoch 00120: val_acc did not improve from 0.88470
196/196 [==============================] - 3

...

195/196 [============================>.] - ETA: 0s - loss: 0.0665 - acc: 0.9760
Epoch 01566: val_acc did not improve from 0.90710
196/196 [==============================] - 35s 177ms/step - loss: 0.0666 - acc: 0.9760 - val_loss: 0.4263 - val_acc: 0.9048
Epoch 1567/10000
195/196 [============================>.] - ETA: 0s - loss: 0.0643 - acc: 0.9767
Epoch 01567: val_acc did not improve from 0.90710
196/196 [==============================] - 35s 181ms/step - loss: 0.0643 - acc: 0.9767 - val_loss: 0.4309 - val_acc: 0.9041
Epoch 1568/10000
195/196 [============================>.] - ETA: 0s - loss: 0.0679 - acc: 0.9760
Epoch 01568: val_acc did not improve from 0.90710
196/196 [==============================] - 35s 180ms/step - loss: 0.0680 - acc: 0.9759 - val_loss: 0.4283 - val_acc: 0.9033
Epoch 1569/10000
195/196 [============================>.] - ETA: 0s - loss: 0.0666 - acc: 0.9757
Epoch 01569: val_acc did not improve from 0.90710
196/196 [==============================] - 35s 177ms/step

Process ForkPoolWorker-40777:
Process ForkPoolWorker-40782:
Process ForkPoolWorker-40783:
Process ForkPoolWorker-40780:
Process ForkPoolWorker-40779:
Process ForkPoolWorker-40788:
Process ForkPoolWorker-40784:
Process ForkPoolWorker-40781:
Process ForkPoolWorker-40778:
Process ForkPoolWorker-40785:
Process ForkPoolWorker-40786:
Process ForkPoolWorker-40787:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/skyolia/anaconda3/envs/tensorflow_gpuenv/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/skyolia/anaconda3/envs/tensorflow_gpuenv/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/skyolia/anaconda3/envs/tensorflow_gpuenv/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap

  File "/home/skyolia/anaconda3/envs/tensorflow_gpuenv/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/home/skyolia/anaconda3/envs/tensorflow_gpuenv/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/home/skyolia/anaconda3/envs/tensorflow_gpuenv/lib/python3.6/multiprocessing/queues.py", line 335, in get
    res = self._reader.recv_bytes()
  File "/home/skyolia/anaconda3/envs/tensorflow_gpuenv/lib/python3.6/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/home/skyolia/anaconda3/envs/tensorflow_gpuenv/lib/python3.6/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/home/skyolia/anaconda3/envs/tensorflow_gpuenv/lib/python3.6/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
  File "/home/skyolia/anaconda3/envs/tensorflow_gpuenv/lib/python3.6/multiprocessing/synchronize.py", line 95, in

Traceback (most recent call last):
  File "/home/skyolia/anaconda3/envs/tensorflow_gpuenv/lib/python3.6/site-packages/tensorflow/python/keras/engine/training_generator.py", line 171, in fit_generator
    x, y, sample_weight=sample_weight, class_weight=class_weight)
  File "/home/skyolia/anaconda3/envs/tensorflow_gpuenv/lib/python3.6/site-packages/tensorflow/python/keras/engine/training.py", line 1828, in train_on_batch
    outputs = self.train_function(ins)
  File "/home/skyolia/anaconda3/envs/tensorflow_gpuenv/lib/python3.6/site-packages/tensorflow/python/keras/backend.py", line 2978, in __call__
    run_metadata=self.run_metadata)
  File "/home/skyolia/anaconda3/envs/tensorflow_gpuenv/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 1399, in __call__
    run_metadata_ptr)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/skyolia/anaconda3/envs/tensorflow_gpuenv/lib/python3.6/s


KeyboardInterrupt

